# User guide

## 数据预处理

患者的纵向生存数据必须被整理成一个2维的，最好为UTF-8编码格式的.csv文件。

这些数据必须是“长格式”（long）的，每行（row）代表某位患者的某一次访视，每列（column）代表不同的数据（如甲胎蛋白，性别，年龄等）。举例：一位患者有三次访视数据，那么在数据库.csv文件中，这位患者的数据将有3行构成，每行各代表其第一，二和三次数据。

额外地，文件必须确保每位患者都拥有至少以下四种数据。这些数据最好一个缺失值也不要有，否则可能会未知地影响程序的运行。这些数据的列名必须与以下列出的名字完全相同（被硬编码入代码中）。

- ‘ID’：每位患者的独有编号。必须注意：这些编号必须是可以被转化成数值的！！！！！！比如，如果ID的值为'EA-01-0001'，则程序会报错；ID为‘1’‘2’则没有问题。附带的R程序中的自建方程toDummy()可以将字符形式的患者编号转化成数字。这个程序会将ID值相等的行归类为同一位患者的不同次访视。

- ‘Time’：此次访视的时间。每位患者的第一次访视（即所谓“基线数据”）的Time都应该等于0；第二及未来访视的Time值为此次访视时间与第一次访视时间的绝对差。例如：访视一为2021年3月1日；访视二为2021年5月1日；若选择Time单位为日，则Time_1 = 0, Time_2 = 51。
    - 为了确保不知名的bug出现，请尽量确保同一名患者不会出现两次时间相同的访视。
    - Time的值是此次访视时间与**基线访视时间**的差，不是与之前一次的！！！
    - 目前不清楚Time的单位的选择是否会对模型的性能有显著影响。目前，在PreCar和PBC队列中，使用月作为基本单位貌似都没有严重的问题。

- ‘Times’：患者的“生存时间”。显而易见地，与ID一样，同一名患者的不同次访视的Times值都应该一样。请确保没有将Times和Time搞混，且两者单位相同。

- ‘Status’：患者在生存时间末端的状态。一般来说，将健康/删失患者标记为0，转癌患者标记为1即可。

其他数据的处理：本代码自带一个将数据进行对数转换（log-transform）的方法。在params.json里声明你想对数转换的变量名称即可。注意，如果名称不出现在cont_list_in里，则不会进行对数转换。目前暂且不支持其它类型的数据转换。